In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import arviz

from typing import Literal
from scipy.stats import spearmanr
from patsy import dmatrix
from cmdstanpy import CmdStanModel, CmdStanMCMC, from_csv

In [43]:
nat_fnames = [
    "naturalness_llama-propbanksenses.csv",
    "naturalness_llama-llamasenses.csv",
    "naturalness_reddit.csv",
    "naturalness_manual.csv"
]

data_nat = []

for fname in nat_fnames:
    df = pd.read_csv(f"../../data/preprocessed/{fname}")

    generation_method = fname.replace("naturalness_", "").strip(".csv")

    if generation_method == "manual":
        df["generation_method_type"] = "manual"
        df["generation_method_subtype"] = "manual"
        df = df.query("sentence_type == 'target'")
        df["generation_method"] = "manual_" + df.naturalness + "_" + df.typicality
    else:
        df["generation_method_type"] = "automated"
        df["generation_method_subtype"] = generation_method
        df["generation_method"] = df[["sentence_type", "naturalness", "typicality"]].agg(
            lambda x: "manual_" + x.naturalness + "_" + x.typicality if x.sentence_type == "calibration" else generation_method,
            axis=1
        )

    data_nat.append(df)

data_nat = pd.concat(data_nat)

data_nat["surprisal_z"] = (data_nat.surprisal - data_nat.surprisal.mean())/data_nat.surprisal.std()
data_nat["freq_z"] = (data_nat.freq - data_nat.freq.mean())/data_nat.freq.std()

if data_nat.sense.isnull().any():
    print("Inserting random dummy senses.")
    data_nat["sense"] = np.random.choice(100, size=data_nat.shape[0]).astype(str)

data_nat

Inserting random dummy senses.


,rater_id,sentence_type,naturalness,typicality,verb,sense,sentence,surprisal,freq,rating,generation_method_type,generation_method_subtype,generation_method,surprisal_z,freq_z
0,618bdf3b-b56d-42e7-8aa5-52111bcb59a9,calibration,unnatural,typical,arrange,90,The baby arranged the something.,196.868866,2289,50,automated,llama-propbanksense,manual_unnatural_typical,0.612365,-0.240193
1,9e5c4371-7b8b-4b63-ba00-b781af055892,calibration,unnatural,typical,arrange,25,The baby arranged the something.,196.868866,2289,0,automated,llama-propbanksense,manual_unnatural_typical,0.612365,-0.240193
2,b6a880d0-6911-4694-a09c-02661b5ca474,calibration,unnatural,typical,arrange,10,The baby arranged the something.,196.868866,2289,22,automated,llama-propbanksense,manual_unnatural_typical,0.612365,-0.240193
3,85ad0f3b-9c2a-40b0-9110-e8251a212b45,calibration,unnatural,typical,arrange,65,The baby arranged the something.,196.868866,2289,0,automated,llama-propbanksense,manual_unnatural_typical,0.612365,-0.240193
4,613f683e-b10c-416c-811e-51b4c7c85889,calibration,unnatural,typical,arrange,6,The baby arranged the something.,196.868866,2289,0,automated,llama-propbanksense,manual_unnatural_typical,0.612365,-0.240193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6815,9421790f-1a80-43e4-8a2a-29a0cb80f28f,target,unnatural,atypical,tear,67,The after tore the brick.,194.907486,1030,25,manual,manual,manual_unnatural_atypical,0.544608,-1.088339
6816,9421790f-1a80-43e4-8a2a-29a0cb80f28f,target,natural,typical,throw,57,The athlete threw the javelin.,220.600510,3679,100,manual,manual,manual_natural_typical,1.432193,0.696203
6817,9421790f-1a80-43e4-8a2a-29a0cb80f28f,target,natural,atypical,throw,91,The jellyfish threw the javelin.,189.328308,3679,92,manual,manual,manual_natural_atypical,0.351871,0.696203
6818,9421790f-1a80-43e4-8a2a-29a0cb80f28f,target,unnatural,typical,throw,74,The athlete threw the while.,224.713104,3679,38,manual,manual,manual_unnatural_typical,1.574265,0.696203


In [44]:
data_nat[["generation_method", "sentence"]].drop_duplicates().generation_method.value_counts().reset_index()
data_nat.groupby("generation_method_subtype").rater_id.value_counts().reset_index().groupby("generation_method_subtype")["count"].value_counts()

generation_method_subtype  count
llama-llamasense           97       76
                           7         1
                           37        1
                           65        1
llama-propbanksense        97       61
manual                     124      55
reddit                     72       60
Name: count, dtype: int64

In [3]:
data_nat.pivot_table(index="generation_method", values="rating", aggfunc=np.mean).sort_values("rating", ascending=False)

/var/folders/0q/s59s3kh51f9gkvj96vpx89bc0000gq/T/ipykernel_19011/2689857857.py:1: FutureWarning: The provided callable <function mean at 0x117002d40> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  data_nat.pivot_table(index="generation_method", values="rating", aggfunc=np.mean).sort_values("rating", ascending=False)


,rating
generation_method,
manual_natural_typical,97.076956
reddit,86.880421
llama-propbanksense,83.418842
manual_natural_atypical,80.671744
llama-llamasense,80.389896
manual_unnatural_typical,19.114020
manual_unnatural_atypical,12.234742


In [4]:
surprisal_freq_corr = spearmanr(data_nat.surprisal, np.log(data_nat.freq))

pval = "< 0. 001" if surprisal_freq_corr.pvalue < 0.001 else "= " + str(surprisal_freq_corr.pvalue)

print(f"\\rho = {round(surprisal_freq_corr.correlation, 2)} (p {pval})")

\rho = -0.1 (p < 0. 001)


In [35]:
typ_fnames = [
    "typicality_llama-propbanksenses.csv",
    "typicality_llama-llamasenses.csv",
    "typicality_reddit.csv",
    "typicality_manual.csv"
]

data_typ = []

for fname in typ_fnames:
    df = pd.read_csv(f"../../data/preprocessed/{fname}")

    generation_method = fname.replace("typicality_", "").strip(".csv")

    if generation_method == "manual":
        df["generation_method_type"] = "manual"
        df = df.query("sentence_type == 'target'")
        df["generation_method"] = "manual_" + df.naturalness + "_" + df.typicality
    else:
        df["generation_method_type"] = "automated"
        df["generation_method_subtype"] = generation_method
        df["generation_method"] = df[["sentence_type", "naturalness", "typicality"]].agg(
            lambda x: "manual_" + x.naturalness + "_" + x.typicality if x.sentence_type == "calibration" else generation_method,
            axis=1
        )

    data_typ.append(df)

data_typ = pd.concat(data_typ)

data_typ["rating_z"] = data_typ.groupby("rater_id").rating.transform(
    lambda x: (x - x.mean())/x.std()
)

data_typ["surprisal_z"] = (data_typ.surprisal - data_typ.surprisal.mean())/data_typ.surprisal.std()
data_typ["freq_z"] = (data_typ.freq - data_typ.freq.mean())/data_typ.freq.std()

if data_typ.sense.isnull().any():
    print("Inserting random dummy senses.")
    data_typ["sense"] = np.random.choice(100, size=data_typ.shape[0]).astype(str)

data_typ

Inserting random dummy senses.


,rater_id,sentence_type,naturalness,typicality,verb,sense,sentence,surprisal,freq,rating,generation_method_type,generation_method,rating_z,surprisal_z,freq_z
0,8ef78292-f35a-4e32-b31e-668c3ebfdfb8,calibration,natural,atypical,drive,54,The chef drove the limo.,161.152435,3239.0,37,automated,manual_natural_atypical,-0.179108,-0.591408,0.348878
1,02f54966-f1b2-42ce-991e-0dd0e084d92a,calibration,natural,atypical,drive,8,The chef drove the limo.,161.152435,3239.0,35,automated,manual_natural_atypical,-0.477253,-0.591408,0.348878
2,3397a8ad-1476-472d-8378-e82303771909,calibration,natural,atypical,drive,75,The chef drove the limo.,161.152435,3239.0,3,automated,manual_natural_atypical,-1.294155,-0.591408,0.348878
3,8e6671ce-296a-4205-8d21-c5161e2f89c5,calibration,natural,atypical,drive,63,The chef drove the limo.,161.152435,3239.0,16,automated,manual_natural_atypical,-1.101403,-0.591408,0.348878
4,ec2bfb5d-e0b9-4ecb-80db-b78c5c039733,calibration,natural,atypical,drive,0,The chef drove the limo.,161.152435,3239.0,51,automated,manual_natural_atypical,-0.257870,-0.591408,0.348878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6723,9b0cf78c-3e69-49f1-8a14-5ed6877cdf50,target,unnatural,atypical,tear,91,The after tore the brick.,194.907486,1030.0,0,manual,manual_unnatural_atypical,-0.640775,0.643788,-1.098299
6724,9b0cf78c-3e69-49f1-8a14-5ed6877cdf50,target,natural,typical,throw,33,The athlete threw the javelin.,220.600510,3679.0,100,manual,manual_natural_typical,1.723989,1.583971,0.637134
6725,9b0cf78c-3e69-49f1-8a14-5ed6877cdf50,target,natural,atypical,throw,12,The jellyfish threw the javelin.,189.328308,3679.0,0,manual,manual_natural_atypical,-0.640775,0.439630,0.637134
6726,9b0cf78c-3e69-49f1-8a14-5ed6877cdf50,target,unnatural,typical,throw,35,The athlete threw the while.,224.713104,3679.0,0,manual,manual_unnatural_typical,-0.640775,1.734462,0.637134


In [45]:
data_typ[["generation_method", "sentence"]].drop_duplicates().generation_method.value_counts().reset_index()
data_typ.groupby("generation_method_subtype").rater_id.value_counts().reset_index().groupby("generation_method_subtype")["count"].value_counts()

KeyError: 'generation_method_subtype'

In [6]:
data_typ_mean = data_typ.pivot_table(
    index="sentence", values="rating_z", aggfunc=np.mean
).rename(
    columns={"rating_z": "typicality_rating"}
).reset_index()

data_typ_mean

/var/folders/0q/s59s3kh51f9gkvj96vpx89bc0000gq/T/ipykernel_19011/1693876360.py:1: FutureWarning: The provided callable <function mean at 0x117002d40> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  data_typ_mean = data_typ.pivot_table(


,sentence,typicality_rating
0,The ALA cast the muppets.,-0.548768
1,The Americans abused the Mexicans.,0.205448
2,The Americans beat the Japanese.,0.611064
3,The BBC covered the inauguration.,1.029906
4,The Baudelaires encountered the unknown.,-0.048643
...,...,...
1835,The youngest met the queen.,0.561288
1836,The youngster pinched the cat.,0.711491
1837,The youngster reached the age.,-0.121412
1838,The youngster saw the girl.,0.771561


In [7]:
data_nat = pd.merge(data_nat, data_typ_mean, how="left")

assert not data_nat.typicality.isnull().any()

data_nat

,rater_id,sentence_type,naturalness,typicality,verb,sense,sentence,surprisal,freq,rating,generation_method_type,generation_method,surprisal_z,freq_z,typicality_rating
0,618bdf3b-b56d-42e7-8aa5-52111bcb59a9,calibration,unnatural,typical,arrange,83,The baby arranged the something.,196.868866,2289,50,automated,manual_unnatural_typical,0.612365,-0.240193,0.087812
1,9e5c4371-7b8b-4b63-ba00-b781af055892,calibration,unnatural,typical,arrange,31,The baby arranged the something.,196.868866,2289,0,automated,manual_unnatural_typical,0.612365,-0.240193,0.087812
2,b6a880d0-6911-4694-a09c-02661b5ca474,calibration,unnatural,typical,arrange,97,The baby arranged the something.,196.868866,2289,22,automated,manual_unnatural_typical,0.612365,-0.240193,0.087812
3,85ad0f3b-9c2a-40b0-9110-e8251a212b45,calibration,unnatural,typical,arrange,37,The baby arranged the something.,196.868866,2289,0,automated,manual_unnatural_typical,0.612365,-0.240193,0.087812
4,613f683e-b10c-416c-811e-51b4c7c85889,calibration,unnatural,typical,arrange,97,The baby arranged the something.,196.868866,2289,0,automated,manual_unnatural_typical,0.612365,-0.240193,0.087812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24533,9421790f-1a80-43e4-8a2a-29a0cb80f28f,target,unnatural,atypical,tear,40,The after tore the brick.,194.907486,1030,25,manual,manual_unnatural_atypical,0.544608,-1.088339,-0.753087
24534,9421790f-1a80-43e4-8a2a-29a0cb80f28f,target,natural,typical,throw,44,The athlete threw the javelin.,220.600510,3679,100,manual,manual_natural_typical,1.432193,0.696203,0.988111
24535,9421790f-1a80-43e4-8a2a-29a0cb80f28f,target,natural,atypical,throw,18,The jellyfish threw the javelin.,189.328308,3679,92,manual,manual_natural_atypical,0.351871,0.696203,-0.633328
24536,9421790f-1a80-43e4-8a2a-29a0cb80f28f,target,unnatural,typical,throw,36,The athlete threw the while.,224.713104,3679,38,manual,manual_unnatural_typical,1.574265,0.696203,-0.786691


In [8]:
data_typ.pivot_table(index="generation_method", values="rating", aggfunc=np.mean).sort_values("rating", ascending=False)

/var/folders/0q/s59s3kh51f9gkvj96vpx89bc0000gq/T/ipykernel_19011/296232787.py:1: FutureWarning: The provided callable <function mean at 0x117002d40> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  data_typ.pivot_table(index="generation_method", values="rating", aggfunc=np.mean).sort_values("rating", ascending=False)


,rating
generation_method,
manual_natural_typical,90.539031
llama-propbanksense,67.651445
reddit,65.832112
llama-llamasense,64.161795
manual_natural_atypical,19.042750
manual_unnatural_typical,16.554331
manual_unnatural_atypical,7.813038


In [9]:
diff_fnames = [
    "difference_llama-propbanksenses.csv",
    "difference_llama-llamasenses.csv",
    "difference_reddit.csv",
    "difference_manual.csv"
]

data_diff = []

for fname in diff_fnames:
    df = pd.read_csv(f"../../data/preprocessed/{fname}")

    df["generation_method"] = fname.replace("difference_", "").strip(".csv")

    data_diff.append(df)

data_diff = pd.concat(data_diff)

data_diff["surprisal1_z"] = (data_diff.surprisal1 - data_diff.surprisal1.mean())/data_diff.surprisal1.std()
data_diff["surprisal2_z"] = (data_diff.surprisal2 - data_diff.surprisal2.mean())/data_diff.surprisal2.std()

data_diff["freq_z"] = (data_diff.freq - data_diff.freq.mean())/data_diff.freq.std()

data_diff["logfreq"] = np.log1p(data_diff["freq"].fillna(0))
data_diff["logfreq_z"] = (data_diff.logfreq - data_diff.logfreq.mean())/data_diff.logfreq.std()

if data_diff.sense1.isnull().any() or data_diff.sense2.isnull().any():
    print("Inserting random dummy senses.")
    data_diff["sense1"] = np.random.choice(100, size=data_diff.shape[0]).astype(str)
    data_diff["sense2"] = np.random.choice(100, size=data_diff.shape[0]).astype(str)

if (data_diff.comparison == "filler").any():
    data_diff_automated_calibration = data_diff[data_diff.pair_type=="filler"].drop(columns=["comparison"])

    data_diff_automated_calibration = pd.merge(
        data_diff_automated_calibration ,
        data_diff[data_diff.generation_method=="manual"][["sentence1", "sentence2", "comparison"]].drop_duplicates()
    )

    data_diff = pd.concat(
        [data_diff[data_diff.pair_type != "filler"], data_diff_automated_calibration],
        axis=0
    )

data_diff

Inserting random dummy senses.


,rater_id,pair_type,comparison,verbnet_class,verb,sense1,sense2,sentence1,sentence2,surprisal1,surprisal2,freq,rating,generation_method,surprisal1_z,surprisal2_z,freq_z,logfreq,logfreq_z
115,3914e1b3-b31c-45c7-9e21-35a89d01491a,target,different,NaN,drop,65,44,The president dropped the subject.,The bank dropped the rate.,181.198166,155.48793,3794,100,llama-propbanksense,0.383227,-0.110381,0.613471,8.241440,0.566663
116,3992d782-59e7-4d88-b1e4-8955fd1fccf1,target,different,NaN,drop,25,33,The president dropped the subject.,The bank dropped the rate.,181.198166,155.48793,3794,14,llama-propbanksense,0.383227,-0.110381,0.613471,8.241440,0.566663
117,4d752c06-d6a8-4f84-8848-40d49dd15909,target,different,NaN,drop,61,84,The president dropped the subject.,The bank dropped the rate.,181.198166,155.48793,3794,100,llama-propbanksense,0.383227,-0.110381,0.613471,8.241440,0.566663
118,a0095902-b95a-4ca1-9c25-256900bd02d0,target,different,NaN,drop,11,56,The president dropped the subject.,The bank dropped the rate.,181.198166,155.48793,3794,98,llama-propbanksense,0.383227,-0.110381,0.613471,8.241440,0.566663
119,a2b742d0-471f-462c-9c6b-ded0c07ee40b,target,different,NaN,drop,44,29,The president dropped the subject.,The bank dropped the rate.,181.198166,155.48793,3794,100,llama-propbanksense,0.383227,-0.110381,0.613471,8.241440,0.566663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,7eefa49c-f6bd-4132-8713-dc0551709e15,filler,same,NaN,line,87,9,The tailor lined the coat.,The insulation lined the attic.,174.965485,207.85396,80,24,reddit,0.171207,1.405655,-1.757541,4.394449,-2.898208
1222,471a3cad-576e-4301-b9d7-7cc3c2cf8256,filler,same,NaN,line,44,63,The tailor lined the coat.,The insulation lined the attic.,174.965485,207.85396,80,6,reddit,0.171207,1.405655,-1.757541,4.394449,-2.898208
1223,11b5969c-37c4-4ae4-9ae7-95918367dda6,filler,same,NaN,line,0,7,The tailor lined the coat.,The insulation lined the attic.,174.965485,207.85396,80,30,reddit,0.171207,1.405655,-1.757541,4.394449,-2.898208
1224,efc18e4c-cbae-4e1e-b099-95d728742ba3,filler,same,NaN,line,46,90,The tailor lined the coat.,The insulation lined the attic.,174.965485,207.85396,80,10,reddit,0.171207,1.405655,-1.757541,4.394449,-2.898208


In [10]:
def bin_response(x: float) -> int:
    if x == 0.0:
        return 1
    elif x == 1.0:
        return 3
    else:
        return 2

def prepare_data(
    data: pd.DataFrame, 
    fixed_formula: str,
    by_subj_formula: str,
    by_verb_formula: str,
    by_sense_formula: str,
    by_item_formula: str,
    item_cols: list[str],
    sense_cols: list[str],
    subj_cols: list[str] = ["rater_id"], 
    verb_cols: list[str] = ["verb"], 
    resp_col: str = "rating",
) -> tuple[dict[str, int | np.ndarray], np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    # scale the responses to [0, 1]
    resp = data[resp_col] / 100

    # bin the responses
    resp_bin = resp.map(bin_response)

    # construct the design matrices
    fixed_predictors = dmatrix(fixed_formula, data, return_type="dataframe")
    by_subj_predictors = dmatrix(by_subj_formula, data, return_type="dataframe")
    by_verb_predictors = dmatrix(by_verb_formula, data, return_type="dataframe")
    by_sense_predictors = dmatrix(by_sense_formula, data, return_type="dataframe")
    by_item_predictors = dmatrix(by_item_formula, data, return_type="dataframe")

    # hash the items and subjects
    subjid = data[subj_cols].agg('-'.join, axis=1).astype("category")
    verbid = data[verb_cols].agg('-'.join, axis=1).astype("category")
    senseid = data[sense_cols].agg('-'.join, axis=1).astype("category")
    itemid = data[item_cols].agg('-'.join, axis=1).astype("category")
    
    # determine whether the subject is a discrete responder
    disc_responder = data[[resp_col]].isin([0, 100])
    disc_responder["subjid"] = subjid
    disc_responder = disc_responder.groupby("subjid")[resp_col].all()

    data_stan = {
        "N_resp": data.shape[0],
        "N_subj": subjid.cat.codes.max() + 1,
        "N_verb": verbid.cat.codes.max() + 1,
        "N_sense": senseid.cat.codes.max() + 1,
        "N_item": itemid.cat.codes.max() + 1,
        "N_fixed": fixed_predictors.shape[1],
        "N_by_subj": by_subj_predictors.shape[1],
        "N_by_verb": by_verb_predictors.shape[1],
        "N_by_sense": by_sense_predictors.shape[1],
        "N_by_item": by_item_predictors.shape[1],
        "fixed_predictors": fixed_predictors.values,
        "by_subj_predictors": by_subj_predictors.values,
        "by_verb_predictors": by_verb_predictors.values,
        "by_sense_predictors": by_sense_predictors.values,
        "by_item_predictors": by_item_predictors.values,
        "subj": subjid.cat.codes.values + 1,
        "verb": verbid.cat.codes.values + 1,
        "sense": senseid.cat.codes.values + 1,
        "item": itemid.cat.codes.values + 1,
        "disc_responder": disc_responder[subjid.cat.categories].values.astype(int),
        "resp_bin": resp_bin.values.astype(int),
        "resp": resp.values,
        "resp_rounded": resp.values.round().astype(int)
    }

    return (
        data_stan, 
        fixed_predictors.columns.values, 
        by_subj_predictors.columns.values,
        by_verb_predictors.columns.values,
        by_sense_predictors.columns.values,
        by_item_predictors.columns.values,
    )


prepare_data(
    data_nat,
    fixed_formula="~ 1",
    by_subj_formula="~ 1",
    by_verb_formula="~ 1",
    by_sense_formula="~ 1",
    by_item_formula="~ 1", 
    item_cols=["sentence"],
    sense_cols=["verb", "sense"],
    subj_cols=["rater_id"] 
)

/var/folders/0q/s59s3kh51f9gkvj96vpx89bc0000gq/T/ipykernel_19011/2098208311.py:44: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disc_responder = disc_responder.groupby("subjid")[resp_col].all()


({'N_resp': 24538,
  'N_subj': 255,
  'N_verb': 95,
  'N_sense': 7794,
  'N_item': 1840,
  'N_fixed': 1,
  'N_by_subj': 1,
  'N_by_verb': 1,
  'N_by_sense': 1,
  'N_by_item': 1,
  'fixed_predictors': array([[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]]),
  'by_subj_predictors': array([[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]]),
  'by_verb_predictors': array([[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]]),
  'by_sense_predictors': array([[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]]),
  'by_item_predictors': array([[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]]),
  'subj': array([ 89, 148, 179, ..., 141, 141, 141], dtype=int16),
  'verb': array([ 4,  4,  4, ..., 92, 92, 92], dtype=int8),
  'sense': array([ 330,  273,  345, ..., 7508, 7527, 755

In [11]:
model = CmdStanModel(stan_file="models/ordered-beta.stan")

In [12]:
def fixed_coef_stats(fit: CmdStanMCMC, coef_names: list[str]) -> pd.DataFrame:
    fixed_coefs = fit.stan_variable("fixed_coefs")

    posterior_means = fixed_coefs.mean(axis=0)
    
    stats = pd.DataFrame(
        np.quantile(fixed_coefs, [0.025, 0.05, 0.95,  0.975], axis=0).T, 
        index=coef_names,
        columns=["2.5%", "5%", "95%", "97.5%"]
    )

    stats["post_mean"] = posterior_means
    
    stats["p"] = np.mean(
        np.sign(posterior_means)[None,:] != np.sign(fixed_coefs), 
        axis=0
    )

    return stats[["post_mean", "2.5%", "5%", "95%", "97.5%", "p"]]

def random_coef_stats(
    fit: CmdStanMCMC, 
    var_name: Literal[
        "subj_cov", "subj_corr", 
        "verb_cov", "verb_corr", 
        "sense_cov", "sense_corr", 
        "item_cov", "item_corr"
    ], 
    coef_names: list[str]
) -> pd.DataFrame:
    matrices = fit.stan_variable(var_name)
    posterior_means = pd.DataFrame(
        matrices.mean(axis=0),
        index=coef_names,
        columns=coef_names
    )

    return posterior_means


def fit_hmc(
    data: pd.DataFrame,
    fixed_formula: str, 
    by_subj_formula: str,
    by_verb_formula: str,
    by_sense_formula: str,
    by_item_formula: str,  
    item_cols: list[str],
    subj_cols: list[str],
    verb_cols: list[str],
    sense_cols: list[str], 
    seed: int = 30298,
    **kwargs
):
    data_stan, fixed_predictors, by_subj_predictors, by_verb_predictors, by_sense_predictors, by_item_predictors = prepare_data(
        data,
        fixed_formula=fixed_formula,
        by_subj_formula=by_subj_formula,
        by_verb_formula=by_verb_formula,
        by_sense_formula=by_sense_formula,
        by_item_formula=by_item_formula, 
        subj_cols=subj_cols,
        verb_cols=verb_cols,
        sense_cols=sense_cols,
        item_cols=item_cols
    )

    fit = model.sample(
        data=data_stan,
        seed=seed,
        **kwargs
    )

    return {
        "fit": fit, 
        "fixed_coefs": fixed_coef_stats(fit, fixed_predictors),
        "subj_cov": random_coef_stats(fit, "subj_cov", by_subj_predictors),
        "verb_cov": random_coef_stats(fit, "verb_cov", by_verb_predictors),
        #"sense_cov": random_coef_stats(fit, "sense_cov", by_sense_predictors),
        "item_cov": random_coef_stats(fit, "item_cov", by_item_predictors),
        "subj_corr": random_coef_stats(fit, "subj_corr", by_subj_predictors),
        "verb_corr": random_coef_stats(fit, "verb_corr", by_verb_predictors),
        #"sense_corr": random_coef_stats(fit, "sense_corr", by_sense_predictors),
        "item_corr": random_coef_stats(fit, "item_corr", by_item_predictors)
    }

def load_fit(
    path: str, 
    data: pd.DataFrame,
    fixed_formula: str, 
    by_subj_formula: str,
    by_verb_formula: str,
    by_sense_formula: str,
    by_item_formula: str,  
    item_cols: list[str],
    subj_cols: list[str],
    verb_cols: list[str],
    sense_cols: list[str],
) -> CmdStanModel:
    _, fixed_predictors, by_subj_predictors, by_verb_predictors, by_sense_predictors, by_item_predictors = prepare_data(
        data,
        fixed_formula=fixed_formula,
        by_subj_formula=by_subj_formula,
        by_verb_formula=by_verb_formula,
        by_sense_formula=by_sense_formula,
        by_item_formula=by_item_formula, 
        subj_cols=subj_cols,
        verb_cols=verb_cols,
        sense_cols=sense_cols,
        item_cols=item_cols
    )

    fit = from_csv(path)

    return {
        "fit": fit, 
        "fixed_coefs": fixed_coef_stats(fit, fixed_predictors),
        "subj_cov": random_coef_stats(fit, "subj_cov", by_subj_predictors),
        "verb_cov": random_coef_stats(fit, "verb_cov", by_verb_predictors),
        #"sense_cov": random_coef_stats(fit, "sense_cov", by_sense_predictors),
        "item_cov": random_coef_stats(fit, "item_cov", by_item_predictors),
        "subj_corr": random_coef_stats(fit, "subj_corr", by_subj_predictors),
        "verb_corr": random_coef_stats(fit, "verb_corr", by_verb_predictors),
        #"sense_corr": random_coef_stats(fit, "sense_corr", by_sense_predictors),
        "item_corr": random_coef_stats(fit, "item_corr", by_item_predictors)
    }

In [13]:
sampler_params = {
    # "adapt_delta": 0.99,
    "iter_warmup": 100, 
    "iter_sampling": 100
}


In [14]:
generation_levels = [
    'manual_natural_typical', 'manual_natural_atypical', 
    'manual_unnatural_typical', 'manual_unnatural_atypical',
    'reddit', 'llama-propbanksense', 'llama-llamasense'
]

In [ ]:
fit_nat = fit_hmc(
    data_nat,
    fixed_formula="~ 1 + C(generation_method, levels=generation_levels)",
    by_subj_formula="~ 1", # cannot fit anything bigger, because not all subjects saw items from every generation method
    by_verb_formula="~ 1", # cannot fit anything bigger, because not all verbs show up with each generation method
    by_sense_formula="~ 1", # cannot fit anything bigger, because not all verb senses show up with each generation method
    by_item_formula="~ 1", # cannot fit anything bigger, because items are specific to generation method
    item_cols = ["sentence"],
    subj_cols = ["rater_id"],
    verb_cols = ["verb"],
    sense_cols = ["verb", "sense"],
    output_dir="fits/nat/base",
    **sampler_params
)

In [ ]:
# fit_nat = load_fit(
#     "fits/nat", 
#     data_nat,
#     fixed_formula="~ 1 + C(generation_method, levels=generation_levels)",
#     by_subj_formula="~ 1", # cannot fit anything bigger, because not all subjects saw items from every generation method
#     by_verb_formula="~ 1", # cannot fit anything bigger, because not all verbs show up with each generation method
#     by_sense_formula="~ 1", # cannot fit anything bigger, because not all verb senses show up with each generation method
#     by_item_formula="~ 1", # cannot fit anything bigger, because items are specific to generation method
#     item_cols = ["sentence"],
#     subj_cols = ["rater_id"],
#     verb_cols = ["verb"],
#     sense_cols = ["verb", "sense"],
# )

In [ ]:
fit_nat["fixed_coefs"]

In [ ]:
fit_nat["fit"]

In [42]:
import re
from typing import Callable

def print_fixed_coef_tabular(fit: CmdStanMCMC, coef_map: Callable[str, str]) -> None:
    print(r"\begin{tabular}{rcccr}")

    print(r"\toprule")
    print(r"      & \textbf{Post. mean} & \textbf{2.5\%} & \textbf{97.5\%} & \textbf{Post.} $p$ \\")
    print(r"\midrule")

    for coef, row in fit["fixed_coefs"].iterrows():
        if row.p < 0.001:
            print(f"{coef_map(coef)} & {row.post_mean:2.2f} & {row['2.5%']:2.2f} & {row['97.5%']:2.2f} & $<$ 0.01 \\\\")
        else:    
            print(f"{coef_map(coef)} & {row.post_mean:2.2f} & {row['2.5%']:2.2f} & {row['97.5%']:2.2f} & {row.p:2.2f} \\\\")

    print(r"\bottomrule")
    print(r"\end{tabular}")

def process_coef_name_nat_typ(coef: str) -> str:
    coef_map = {
        "manual_unnatural_typical": "Manual (Unnatural \\& Typical)",
        "manual_natural_typical": "Manual (Natural \\& Typical )",
        "manual_unnatural_atypical": "Manual (Unnatural \\& Atypical)",
        "manual_natural_atypical": "Manual (Natural \\& Atypical)",
        "reddit": "Corpus",
        "llama-propbanksense": "LM with PropBank senses",
        "llama-llamasense": "LM with LM senses"
    }

    if coef == "Intercept":
        return coef
    else:
        return " $\\times$ ".join(
            coef_map[v] for v in re.findall(
                "C\(generation_method, levels=generation_levels\)\[T\.(.*)\]", 
                coef
            )
        )

In [ ]:
              
print_fixed_coef_tabular(fit_nat, process_coef_name_nat_typ)

In [ ]:
_ = arviz.plot_posterior(fit_nat["fit"], var_names=["subj_scale", "item_scale", "verb_scale"])

In [ ]:
cutpoint0_nat = fit_nat["fit"].stan_variable("cutpoint0")

print(
    np.round(cutpoint0_nat.mean(0), 2), 
    f"[{np.round(np.quantile(cutpoint0_nat, 0.025), 2)}, {np.round(np.quantile(cutpoint0_nat, 0.975), 2)}]"
)

In [ ]:
cutpoint1_nat = cutpoint0_nat + np.exp(fit_nat["fit"].stan_variable("interval_size_logmean"))

print(
    np.round(cutpoint1_nat.mean(0), 2), 
    f"[{np.round(np.quantile(cutpoint1_nat, 0.025), 2)}, {np.round(np.quantile(cutpoint1_nat, 0.975), 2)}]"
)

In [ ]:
((fit_nat["fit"].stan_variable("fixed_coefs")[:,4:] - fit_nat["fit"].stan_variable("fixed_coefs")[:,[1]]) > 0).mean(0)

In [ ]:
ax = sns.histplot(fit_nat["fit"].stan_variable("prediction").mean(0))
ax.axvline(x=fit_nat["fit"].stan_variable("cutpoint0").mean(0), color="orange")
ax.axvline(x=np.exp(fit_nat["fit"].stan_variable("interval_size_logmean")).mean(0), color="orange")

In [ ]:
sns.histplot(fit_nat["fit"].stan_variable("interval_mean").mean(0))
sns.histplot(data_nat[(data_nat.rating > 0) & (data_nat.rating < 100)].rating/100)

In [56]:
fit_nat_typicality = fit_hmc(
    data_nat,
    fixed_formula="~ 1 + typicality_rating",
    by_subj_formula="~ 1 + typicality_rating",
    by_verb_formula="~ 1 + typicality_rating",
    by_sense_formula="~ 1 + typicality_rating", 
    by_item_formula="~ 1", # cannot fit anything bigger, because each sentence has only one typicality rating
    item_cols = ["sentence"],
    subj_cols = ["rater_id"],
    verb_cols = ["verb"],
    sense_cols = ["verb", "sense"],
    output_dir="fits/nat/typicality",
    **sampler_params
)

/var/folders/0q/s59s3kh51f9gkvj96vpx89bc0000gq/T/ipykernel_85606/2098208311.py:44: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disc_responder = disc_responder.groupby("subjid")[resp_col].all()
17:01:27 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

06:10:44 - cmdstanpy - INFO - CmdStan done processing.
06:10:44 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/Users/awhite48/Projects/item-validation/scripts/analysis/models/ordered-beta.stan', line 123, column 2 to column 26)
	Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/Users/awhite48/Projects/item-validation/scripts/analysis/models/ordered-beta.stan', line 123, column 2 to column 26)
	Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/Users/awhite48/Projects/item-validation/scripts/analysis/models/ordered-beta.stan', line 135, column 2 to column 26)
	Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/Users/awhite48/Projects/item-validation/scripts/analysis/models/ordered-beta.stan', line 135, column 2 to column 26)
	Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/Users/awhite48/Project

In [58]:
fit_nat_typicality["fixed_coefs"]

,post_mean,2.5%,5%,95%,97.5%,p
Intercept,1.015936,0.896511,0.913489,1.115839,1.129717,0.0
typicality_rating,1.507075,1.382702,1.401676,1.607654,1.633840,0.0


In [65]:
fit_nat_typicality["subj_cov"]

,Intercept,typicality_rating
Intercept,0.401458,-0.039988
typicality_rating,-0.039988,0.366253


In [ ]:
fit_nat_freq_surprisal["fixed_coefs"]

In [ ]:
fit_typ = fit_hmc(
    data_typ,
    fixed_formula="~ 1 + C(generation_method, levels=generation_levels)",
    by_subj_formula="~ 1", # cannot fit anything bigger, because not all subjects saw items from every generation method
    by_verb_formula="~ 1", # cannot fit anything bigger, because not all verbs show up with each generation method
    by_sense_formula="~ 1", # cannot fit anything bigger, because not all verb senses show up with each generation method
    by_item_formula="~ 1", # cannot fit anything bigger, because items are specific to generation method
    item_cols = ["sentence"],
    subj_cols = ["rater_id"],
    verb_cols = ["verb"],
    sense_cols = ["verb", "sense"],
    output_dir="fits/typ/base/",
    **sampler_params
)

In [ ]:
fit_typ["fixed_coefs"]

In [ ]:
print_fixed_coef_tabular(fit_typ, process_coef_name_nat_typ)

In [ ]:
cutpoint0_typ = fit_typ["fit"].stan_variable("cutpoint0")

print(
    np.round(cutpoint0_typ.mean(0), 2), 
    f"[{np.round(np.quantile(cutpoint0_typ, 0.025), 2)}, {np.round(np.quantile(cutpoint0_typ, 0.975), 2)}]"
)

In [ ]:
cutpoint1_typ = cutpoint0_typ + np.exp(fit_typ["fit"].stan_variable("interval_size_logmean"))

print(
    np.round(cutpoint1_typ.mean(0), 2), 
    f"[{np.round(np.quantile(cutpoint1_typ, 0.025), 2)}, {np.round(np.quantile(cutpoint1_typ, 0.975), 2)}]"
)

In [ ]:
_ = arviz.plot_posterior(fit_typ["fit"], var_names=["fixed_coefs"])

In [ ]:
_ = arviz.plot_posterior(fit_typ["fit"], var_names=["subj_scale", "item_scale", "verb_scale"])

In [ ]:

_ = arviz.plot_posterior(fit_typ["fit"], var_names=["sample_size", "interval_size"])

In [19]:
fit_typ_surprisal_add = fit_hmc(
    data_typ,
    fixed_formula="~ 1 + C(generation_method, levels=generation_levels) + surprisal_z",
    by_subj_formula="~ 1 + surprisal_z", # cannot fit anything bigger, because not all subjects saw items from every generation method
    by_verb_formula="~ 1 + surprisal_z", # cannot fit anything bigger, because not all verbs show up with each generation method
    by_sense_formula="~ 1 + surprisal_z", # cannot fit anything bigger, because not all verb senses show up with each generation method
    by_item_formula="~ 1", # cannot fit anything bigger, because items are specific to generation method
    item_cols = ["sentence"],
    subj_cols = ["rater_id"],
    verb_cols = ["verb"],
    sense_cols = ["verb", "sense"],
    output_dir="fits/typ/surprisal/add/",
    **sampler_params
)

/var/folders/0q/s59s3kh51f9gkvj96vpx89bc0000gq/T/ipykernel_19011/2098208311.py:44: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disc_responder = disc_responder.groupby("subjid")[resp_col].all()
08:46:29 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

In [15]:
fit_typ_surprisal_inter = fit_hmc(
    data_typ,
    fixed_formula="~ 1 + C(generation_method, levels=generation_levels) * surprisal_z",
    by_subj_formula="~ 1 + surprisal_z", # cannot fit anything bigger, because not all subjects saw items from every generation method
    by_verb_formula="~ 1 + surprisal_z", # cannot fit anything bigger, because not all verbs show up with each generation method
    by_sense_formula="~ 1 + surprisal_z", # cannot fit anything bigger, because not all verb senses show up with each generation method
    by_item_formula="~ 1", # cannot fit anything bigger, because items are specific to generation method
    item_cols = ["sentence"],
    subj_cols = ["rater_id"],
    verb_cols = ["verb"],
    sense_cols = ["verb", "sense"],
    output_dir="fits/typ/surprisal/inter/",
    **sampler_params
)

/var/folders/0q/s59s3kh51f9gkvj96vpx89bc0000gq/T/ipykernel_19011/2098208311.py:44: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disc_responder = disc_responder.groupby("subjid")[resp_col].all()
06:54:23 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

08:01:39 - cmdstanpy - INFO - CmdStan done processing.
08:01:39 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/Users/awhite48/Projects/item-validation/scripts/analysis/models/ordered-beta.stan', line 123, column 2 to column 26)
	Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/Users/awhite48/Projects/item-validation/scripts/analysis/models/ordered-beta.stan', line 123, column 2 to column 26)
	Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/Users/awhite48/Projects/item-validation/scripts/analysis/models/ordered-beta.stan', line 123, column 2 to column 26)
	Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/Users/awhite48/Projects/item-validation/scripts/analysis/models/ordered-beta.stan', line 123, column 2 to column 26)
	Exception: ordered_logistic: Cut-points is not a valid ordered vector. The element at 2 is -8.00995

08:03:29 - cmdstanpy - WARNING - Some chains may have failed to converge.
	Chain 2 had 1 iterations at max treedepth (1.0%)
	Use the "diagnose()" method on the CmdStanMCMC object to see further information.


In [16]:
fit_typ_surprisal["fixed_coefs"]

,post_mean,2.5%,5%,95%,97.5%,p
Intercept,1.767029,1.401943,1.448645,2.074480,2.126637,0.0000
"C(generation_method, levels=generation_levels)[T.manual_natural_atypical]",-3.376138,-3.842738,-3.782433,-2.934457,-2.874336,0.0000
"C(generation_method, levels=generation_levels)[T.manual_unnatural_typical]",-3.568834,-3.961968,-3.904942,-3.194973,-3.106191,0.0000
"C(generation_method, levels=generation_levels)[T.manual_unnatural_atypical]",-4.259769,-4.705152,-4.654186,-3.829644,-3.771394,0.0000
"C(generation_method, levels=generation_levels)[T.reddit]",-1.315255,-1.723059,-1.648007,-0.960300,-0.858022,0.0000
"C(generation_method, levels=generation_levels)[T.llama-propbanksense]",-1.348004,-1.766141,-1.700176,-1.012774,-0.926953,0.0000
"C(generation_method, levels=generation_levels)[T.llama-llamasense]",-1.470662,-1.871678,-1.822997,-1.147207,-1.049944,0.0000
surprisal_z,-0.139786,-0.546132,-0.470923,0.181955,0.259596,0.2600
"C(generation_method, levels=generation_levels)[T.manual_natural_atypical]:surprisal_z",0.023273,-0.502292,-0.384820,0.467407,0.559133,0.4875
"C(generation_method, levels=generation_levels)[T.manual_unnatural_typical]:surprisal_z",0.126696,-0.315990,-0.280452,0.490033,0.598886,0.2800


In [17]:
fit_typ_freq = fit_hmc(
    data_typ,
    fixed_formula="~ 1 + C(generation_method, levels=generation_levels) * freq_z",
    by_subj_formula="~ 1 + freq_z", # cannot fit anything bigger, because not all subjects saw items from every generation method
    by_verb_formula="~ 1", # cannot fit anything bigger, because not all verbs show up with each generation method
    by_sense_formula="~ 1", # cannot fit anything bigger, because not all verb senses show up with each generation method
    by_item_formula="~ 1", # cannot fit anything bigger, because items are specific to generation method
    item_cols = ["sentence"],
    subj_cols = ["rater_id"],
    verb_cols = ["verb"],
    sense_cols = ["verb", "sense"],
    output_dir="fits/typ/freq",
    **sampler_params
)

/var/folders/0q/s59s3kh51f9gkvj96vpx89bc0000gq/T/ipykernel_19011/2098208311.py:44: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disc_responder = disc_responder.groupby("subjid")[resp_col].all()
08:41:58 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

KeyboardInterrupt: 

In [38]:
generation_levels = [
    'manual', 'reddit', 'llama-propbanksense', 'llama-llamasense'
]

comparison_levels = ['same', 'different']

fit_pairdiff = fit_hmc(
    data_diff,
    fixed_formula="~ 1 + C(comparison, levels=comparison_levels) * C(generation_method, levels=generation_levels)",
    by_subj_formula="~ 1 + C(comparison, levels=comparison_levels)",
    by_verb_formula="~ 1 + C(comparison, levels=comparison_levels)",
    by_sense_formula="~ 1", 
    by_item_formula="~ 1",
    item_cols=["sentence1", "sentence2"],
    sense_cols = ["verb", "sense1", "sense2"],
    subj_cols=["rater_id"],
    verb_cols = ["verb"],
    output_dir="fits/pairdiff/base/",
    **sampler_params
)

/var/folders/0q/s59s3kh51f9gkvj96vpx89bc0000gq/T/ipykernel_85606/2098208311.py:44: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disc_responder = disc_responder.groupby("subjid")[resp_col].all()
16:03:58 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

16:23:36 - cmdstanpy - INFO - CmdStan done processing.
16:23:36 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/Users/awhite48/Projects/item-validation/scripts/analysis/models/ordered-beta.stan', line 123, column 2 to column 26)
	Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/Users/awhite48/Projects/item-validation/scripts/analysis/models/ordered-beta.stan', line 123, column 2 to column 26)
	Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/Users/awhite48/Projects/item-validation/scripts/analysis/models/ordered-beta.stan', line 123, column 2 to column 26)
	Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/Users/awhite48/Projects/item-validation/scripts/analysis/models/ordered-beta.stan', line 123, column 2 to column 26)
	Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/Users/awhite48/Project

In [39]:
fit_pairdiff["fixed_coefs"]

,post_mean,2.5%,5%,95%,97.5%,p
Intercept,-1.224552,-1.606147,-1.542929,-0.925876,-0.887493,0.0000
"C(comparison, levels=comparison_levels)[T.different]",1.500223,1.188013,1.228454,1.761293,1.824804,0.0000
"C(generation_method, levels=generation_levels)[T.reddit]",0.190702,-0.278832,-0.235914,0.693218,0.797845,0.2750
"C(generation_method, levels=generation_levels)[T.llama-propbanksense]",-0.361401,-0.924657,-0.830297,0.157733,0.282955,0.1150
"C(generation_method, levels=generation_levels)[T.llama-llamasense]",-0.072489,-0.513787,-0.477690,0.318860,0.375127,0.3625
"C(comparison, levels=comparison_levels)[T.different]:C(generation_method, levels=generation_levels)[T.reddit]",-0.357808,-0.725857,-0.680868,-0.033463,0.002347,0.0275
"C(comparison, levels=comparison_levels)[T.different]:C(generation_method, levels=generation_levels)[T.llama-propbanksense]",0.049226,-0.412897,-0.282230,0.338880,0.414518,0.3850
"C(comparison, levels=comparison_levels)[T.different]:C(generation_method, levels=generation_levels)[T.llama-llamasense]",-0.367102,-0.696020,-0.659104,-0.100192,0.005753,0.0300


In [44]:
def process_coef_name_pairdiff(coef: str) -> str:
    coef_map = {
        "manual": "Manual",
        "reddit": "Corpus",
        "llama-propbanksense": "LM with PropBank senses",
        "llama-llamasense": "LM with LM senses",
        "different": "Different Sense",
        "same": "Same Sense"
    }

    if coef == "Intercept":
        return coef
    else:
        return " $\\times$ ".join(
            coef_map[v] for v in re.findall(
                "C\(.*?\)\[T\.(.*?)\]", 
                coef
            )
        )

print_fixed_coef_tabular(fit_pairdiff, process_coef_name_pairdiff)

\begin{tabular}{rcccr}
\toprule
      & \textbf{Post. mean} & \textbf{2.5\%} & \textbf{97.5\%} & \textbf{Post.} $p$ \\
\midrule
Intercept & -1.22 & -1.61 & -0.89 & $<$ 0.01 \\
Different Sense & 1.50 & 1.19 & 1.82 & $<$ 0.01 \\
Corpus & 0.19 & -0.28 & 0.80 & 0.28 \\
LM with PropBank senses & -0.36 & -0.92 & 0.28 & 0.12 \\
LM with LM senses & -0.07 & -0.51 & 0.38 & 0.36 \\
Different Sense $\times$ Corpus & -0.36 & -0.73 & 0.00 & 0.03 \\
Different Sense $\times$ LM with PropBank senses & 0.05 & -0.41 & 0.41 & 0.39 \\
Different Sense $\times$ LM with LM senses & -0.37 & -0.70 & 0.01 & 0.03 \\
\bottomrule
\end{tabular}


In [45]:
cutpoint0_pairdiff = fit_pairdiff["fit"].stan_variable("cutpoint0")

print(
    np.round(cutpoint0_pairdiff.mean(0), 2), 
    f"[{np.round(np.quantile(cutpoint0_pairdiff, 0.025), 2)}, {np.round(np.quantile(cutpoint0_pairdiff, 0.975), 2)}]"
)

-2.51 [-2.59, -2.43]


In [46]:
cutpoint1_pairdiff = cutpoint0_pairdiff + np.exp(fit_pairdiff["fit"].stan_variable("interval_size_logmean"))

print(
    np.round(cutpoint1_pairdiff.mean(0), 2), 
    f"[{np.round(np.quantile(cutpoint1_pairdiff, 0.025), 2)}, {np.round(np.quantile(cutpoint1_pairdiff, 0.975), 2)}]"
)

1.08 [0.46, 1.82]


In [ ]:
_ = arviz.plot_posterior(fit_pairdiff["fit"], var_names=["fixed_coefs"])

In [ ]:
_ = arviz.plot_posterior(fit_pairdiff["fit"], var_names=["subj_scale", "item_scale", "verb_scale"])

In [ ]:

_ = arviz.plot_posterior(fit_pairdiff["fit"], var_names=["sample_size", "interval_size"])

In [ ]:
fit_pairdiff_surprisal = fit_hmc(
    data_diff,
    fixed_formula=("~ 1 + C(PairType, levels=comparison_levels) * surprisal1_z * surprisal2_z"
                      " + C(PairType, levels=comparison_levels) * typicality1 * typicality2" 
                      " + C(PairType, levels=comparison_levels) * freq"),
    by_subj_formula="~ 1 + C(PairType, levels=comparison_levels)",
    by_verb_formula="~ 1 + C(PairType, levels=comparison_levels)",
    by_sense_formula="~ 1", 
    by_item_formula="~ 1", 
    max_init_iter=0,
    item_cols=["sentence1", "sentence2"],
    sense_cols = ["verb", "sense1", "sense2"],
    subj_cols=["rater_id"],
    verb_cols = ["verb"],
    output_dir="fits/pairdiff/",
    **sampler_params
)

In [ ]:
fit_pairdiff_surprisal["fixed_coefs"]

In [ ]:
print_fixed_coef_tabular(fit_pairdiff_surprisal, process_coef_name_pairdiff_surprisal)

In [ ]:
_ = arviz.plot_posterior(fit_pairdiff_surprisal["fit"], var_names=["fixed_coefs"])

In [ ]:
_ = arviz.plot_posterior(fit_pairdiff_surprisal["fit"], var_names=["subj_scale", "item_scale", "verb_scale"])

In [ ]:

_ = arviz.plot_posterior(fit_pairdiff_surprisal["fit"], var_names=["sample_size", "interval_size"])